In [125]:
from tabulate import tabulate

In [127]:
available_registers=["R0","R1"]
register_status={reg:None for reg in available_registers}
usage_count={reg: 0 for reg in available_registers}  
address_descriptor={}  
instruction_costs={"MOV":1,"ADD":3,"SUB":3}  

In [129]:
def getreg():
    for reg,value in register_status.items():
        if value is None:
            return reg
    least_used_reg=min(usage_count,key=usage_count.get)
    register_status[least_used_reg]=None
    return least_used_reg

In [131]:
def free_reg(reg):
    if reg in register_status:
        register_status[reg]=None
        usage_count[reg]=0

In [155]:
def generate_code(statements):
    table_data=[]
    reg_map={}
    instruction_list=[]
    total_cost=0
    for stmt in statements:
        dest,expr=stmt.split("=")
        operands=expr.split(" ")
        reg=getreg()     
        register_status[reg]=dest  
        reg_map[dest]=reg
        usage_count[reg]+=1 
        address_descriptor[dest]=f"{dest} in {reg}"        
        instructions=[]
        cost=0       
        if operands[1]=="+":
            instructions.append(f"MOV {operands[0]},{reg}")
            cost+=instruction_costs["MOV"]
            instructions.append(f"ADD {operands[2]},{reg}")
            cost+=instruction_costs["ADD"]
        elif operands[1]=="-":
            instructions.append(f"MOV {operands[0]},{reg}")
            cost+=instruction_costs["MOV"]
            instructions.append(f"SUB {operands[2]},{reg}")
            cost+=instruction_costs["SUB"]        
        instruction_list.extend(instructions)        
        table_data.append([stmt,reg,"\n".join(instructions),cost,f"{reg} holds {dest}",address_descriptor[dest]])
        total_cost+=cost       
    headers=["Statement","Register","Instructions","Cost","Register Descriptor","Address Descriptor"]
    print(tabulate(table_data,headers,tablefmt="grid"))
    print("\nTotal Cost:",total_cost)

In [157]:
statements=["T1 = a + b","T2 = c + d","T3 = e - T2","T4 = T1 - T3"]
generate_code(statements)

+--------------+------------+----------------+--------+-----------------------+----------------------+
| Statement    | Register   | Instructions   |   Cost | Register Descriptor   | Address Descriptor   |
+==============+============+================+========+=======================+======================+
| T1 = a + b   | R0         | MOV a, R0      |      4 | R0 holds T1           | T1 in R0             |
|              |            | ADD b, R0      |        |                       |                      |
+--------------+------------+----------------+--------+-----------------------+----------------------+
| T2 = c + d   | R1         | MOV c, R1      |      4 | R1 holds T2           | T2 in R1             |
|              |            | ADD d, R1      |        |                       |                      |
+--------------+------------+----------------+--------+-----------------------+----------------------+
| T3 = e - T2  | R0         | MOV e, R0      |      4 | R0 holds T3      